In [1]:
import pandas as pd
import os
from selenium import webdriver
import time
import re # 정규식을 활용하여 괄호 안 숫자 추출

#os.chdir("")

In [2]:
driver = webdriver.Chrome("chromedriver")
driver.maximize_window() # 창 크기 최대화

rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])
user_review_id = {}
review_json = {}
image_json = {}
item_meta = {}
items = ["공릉순두부","공릉역 찬이네"]
for item in items:
    driver.get("https://www.google.com/maps/") # 구글 지도 접속하기
    searchbox = driver.find_element_by_css_selector("input#searchboxinput") # 검색창에 입력하기
    searchbox.send_keys(item)
    searchbutton = driver.find_element_by_class_name("searchbox-searchbutton") # 검색버튼 누르기
    searchbutton.click()
    time.sleep(5)
    reviewbutton = driver.find_element_by_class_name("widget-pane-link")
    res_content = driver.find_element_by_xpath("//button[@jsaction='pane.rating.category']").text
    reviewbutton.click()
    time.sleep(10)

    # 리뷰 전체 뽑기 위해 스크롤 다운
    while True:
        try:
            driver.find_element_by_class_name('section-loading').click()
            time.sleep(2)
        except:
            print(item + " 스크롤 다운 완료")
            break

    spread_review = driver.find_elements_by_xpath("//button[@jsaction='pane.review.expandReview']") 
    for i in range(len(spread_review)): 
        isTrue = spread_review[i].is_displayed() # 보이는 것인지를 확인 
        print("Element is visible? " + str(isTrue)) 
        if isTrue: 
            spread_review[i].click() 
            print(str(i)+"th more button is clicked and wait 2 secs...") 
            time.sleep(2)
        else:
            print("Element is visible? " + str(isTrue)) 


    # 시간 지연
    time.sleep(3)
    user = driver.find_elements_by_xpath("//a[@class='section-review-reviewer-link']")
    user_review = driver.find_elements_by_xpath("//button[@class='section-review-action-menu']")
    date = driver.find_elements_by_xpath("//span[@class='section-review-publish-date']")
    rating = driver.find_elements_by_xpath("//span[@class='section-review-stars']")

    time.sleep(5)
    review = driver.find_elements_by_xpath("//span[@class='section-review-text']")


    # Image

    image = driver.find_elements_by_xpath("//button[@class='section-review-photo']")
    review_json[item] = {}
    user_review_id[item] = {}
    image_json[item] = {}
    item_meta[item] = {"category" : res_content}
    for i in range(len(date)):
        if review[i].text is not "":
            user_elem = re.findall("\d+",user[i].get_attribute("href"))[0]
            item_elem = item
            rating_elem = re.findall("\d+",rating[i].get_attribute('aria-label'))[0]
            date_elem = date[i].text
            row = {"UserID":user_elem,"ItemID":item_elem,"Rating":rating_elem,"Timestamp":date_elem}
            row = pd.DataFrame(row, index=[i])
            rating_df = rating_df.append(row,ignore_index=True)
            user_review_id[item][user_elem] = user_review[i].get_attribute("data-review-id")
            review_json[item][user_review[i].get_attribute("data-review-id")] = review[i].text
    for i in range(len(image)):
        image_json[item][image[i].get_attribute("data-review-id")] = re.findall(r'\"(.+?)\"',image[i].get_attribute('style'))[0]

# 크롬창 닫기
driver.close()

공릉순두부 스크롤 다운 완료
공릉역 찬이네 스크롤 다운 완료


In [3]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,118420402434392385647,공릉순두부,5,4달 전
1,107313277775324604933,공릉순두부,4,10달 전
2,107285647276587086824,공릉순두부,5,1달 전
3,103086684508426392657,공릉순두부,4,1달 전
4,106158307773488526036,공릉순두부,3,5일 전
5,107090457212642993215,공릉순두부,5,9달 전
6,108694562437481047860,공릉순두부,4,3달 전
7,116018171918130152048,공릉순두부,3,4달 전
8,114306534734234679109,공릉순두부,3,3달 전
9,109568538346691003037,공릉순두부,5,1년 전


In [4]:
user_review_id

{'공릉순두부': {'118420402434392385647': 'ChZDSUhNMG9nS0VJQ0FnSUNpdXFuVVJREAE',
  '107313277775324604933': 'ChZDSUhNMG9nS0VJQ0FnSUNjNV9iak5BEAE',
  '107285647276587086824': 'ChZDSUhNMG9nS0VJQ0FnSUN5Nk1lTFRREAE',
  '103086684508426392657': 'ChdDSUhNMG9nS0VJQ0FnSUN5d2EzWGdRRRAB',
  '106158307773488526036': 'ChdDSUhNMG9nS0VJQ0FnSUR5dnZPZzh3RRAB',
  '107090457212642993215': 'ChdDSUhNMG9nS0VJQ0FnSURjX1BXRXlBRRAB',
  '108694562437481047860': 'ChZDSUhNMG9nS0VJQ0FnSURpMks3a1VREAE',
  '116018171918130152048': 'ChZDSUhNMG9nS0VJQ0FnSUNpeHJPd2RREAE',
  '114306534734234679109': 'ChZDSUhNMG9nS0VJQ0FnSURpeE1ubENBEAE',
  '109568538346691003037': 'ChdDSUhNMG9nS0VJQ0FnSUQwMGJDcnF3RRAB',
  '100562813415515374381': 'ChdDSUhNMG9nS0VJQ0FnSURzX3JyM3pRRRAB',
  '109176744319737408266': 'ChdDSUhNMG9nS0VJQ0FnSURjaE4yM193RRAB',
  '109084374051453599882': 'ChZDSUhNMG9nS0VJQ0FnSURNdTdpR1hREAE',
  '106807338538568445001': 'ChdDSUhNMG9nS0VJQ0FnSUQwa1pfSXdnRRAB',
  '104569519319113667167': 'ChdDSUhNMG9nS0VJQ0FnSUQwa2VfR2tB

In [5]:
review_json

{'공릉순두부': {'ChZDSUhNMG9nS0VJQ0FnSUNpdXFuVVJREAE': '반찬이 집반찬처럼 맛있게나온다.\n그냥 순두부가 제일 괜찮다.\n맛있고 저렴한 밥집.',
  'ChZDSUhNMG9nS0VJQ0FnSUNjNV9iak5BEAE': '우삼겹순두부찌개 완전 맛남!\n제육은 솔직히 그냥그랬고\n순두부찌개 강추!!\n밥도 무한리필이고\n같이나온 기본  찬들도 맛있었당ㅋㅋ',
  'ChZDSUhNMG9nS0VJQ0FnSUN5Nk1lTFRREAE': '이정도면 5점을 줘도 괜찮을거 같네요',
  'ChdDSUhNMG9nS0VJQ0FnSUN5d2EzWGdRRRAB': '햄순두부 맛있네요',
  'ChdDSUhNMG9nS0VJQ0FnSUR5dnZPZzh3RRAB': '해물이 오래된것 같았다.',
  'ChdDSUhNMG9nS0VJQ0FnSURjX1BXRXlBRRAB': '이 주변에서 점심먹기 좋은 장소 중 한곳입니다. 적당한 가격에 다양한 퓨전순두부 요리로, 매일 점심마다 사람이 붐빕니다. 12시 이후에 가면 기다려야합니다.',
  'ChZDSUhNMG9nS0VJQ0FnSURpMks3a1VREAE': '가성비좋은순두부',
  'ChZDSUhNMG9nS0VJQ0FnSUNpeHJPd2RREAE': '서빙하는 분이 불친절하시네요',
  'ChZDSUhNMG9nS0VJQ0FnSURpeE1ubENBEAE': '가성비로는 괜찮음',
  'ChdDSUhNMG9nS0VJQ0FnSUQwMGJDcnF3RRAB': '갈때마다 집밥을 먹는 듯.\n정성듬뿍 반찬과 가벼운 주머니가 무색하게 풍성한 양, 강력 추천합니다.',
  'ChdDSUhNMG9nS0VJQ0FnSURzX3JyM3pRRRAB': '차돌 된장찌개 밥도둑~',
  'ChdDSUhNMG9nS0VJQ0FnSURjaE4yM193RRAB': '음식이넘 .♡\n깔끔해유  집밥처럼♡♡♡♡♡',
  'ChZDSUhNMG9nS0VJQ0FnSURNdTdpR1hREAE': '(Google 번역 제공) 순두부를 좋아하고 다양한 종

In [6]:
image_json

{'공릉순두부': {'ChZDSUhNMG9nS0VJQ0FnSUNjNV9iak5BEAE': 'https://lh5.googleusercontent.com/p/AF1QipPU-lkOb6DNo3APm8f68UcoRHbTjaGE1JYcZmrW=w750-h750-p-k-no',
  'ChdDSUhNMG9nS0VJQ0FnSUN5d2EzWGdRRRAB': 'https://lh5.googleusercontent.com/p/AF1QipPAC1Z5tpUsPVLw2efnKjdDz8vWpv2I1U2XAxdm=w750-h750-p-k-no',
  'ChdDSUhNMG9nS0VJQ0FnSUNNX2VUc25nRRAB': 'https://lh5.googleusercontent.com/p/AF1QipPNbnOkJGQx9U36A9_MksZMMeR9xb07en_sX1Bx=w750-h750-p-k-no',
  'ChZDSUhNMG9nS0VJQ0FnSUNpNkx6cmZnEAE': 'https://lh5.googleusercontent.com/p/AF1QipP1PKrSLTUQJ5wVXGnKCE3txIYbUKOfBiMindj3=w750-h750-p-k-no'},
 '공릉역 찬이네': {'ChZDSUhNMG9nS0VJQ0FnSUNzMDY3WktREAE': 'https://lh5.googleusercontent.com/p/AF1QipM3ffvlEOl1mjN_dPALnawMHpk7buerWH8ONCLh=w750-h750-p-k-no',
  'ChZDSUhNMG9nS0VJQ0FnSURjNkphQ1lnEAE': 'https://lh5.googleusercontent.com/p/AF1QipM0On4K7JlL6NqdBwjD2Zx0dyTNFJ7ZdTQ_h5eS=w750-h750-p-k-no',
  'ChZDSUhNMG9nS0VJQ0FnSURZb3BTQmZBEAE': 'https://lh5.googleusercontent.com/p/AF1QipMBBQ-klGVXz-T9no7hdy1mX0FDFt8P2uau44TY=w7

In [7]:
item_meta

{'공릉순두부': {'category': '음식점'}, '공릉역 찬이네': {'category': '음식점'}}